Задача: прогнозирование нагруженности площадок

Прогноз нагруженности площадок: cобирает актуальные данные для предоставления администрации города оптимальных способов развития спортивной инфраструктуры, повышает пользовательский опыт.

In [1]:
!pip install numpy==1.26.4
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\Vadim\\AppData\\Roaming\\Python\\Python311\\site-packages\\~-mpy.libs\\libscipy_openblas64_-c16e4918366c6bc1f1cd71e28ca36fc0.dll'
Check the permissions.



Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for numpy==1.26.4 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 320.0 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.0 kB 217.9 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.0 kB 290.5 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 270.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB 1.9 MB/s eta 0:00:09
   ---------------------------------------- 0.1/15.8 MB 787.7 kB/s eta 0:00:20
   ---------------------------------------- 0.1/15.8 MB 1.0 MB/s eta 0:00:16
   -----------

In [8]:
df = pd.read_csv("grounds_data_120.csv")
df.head()

,hour,day,month,year,ground_id,is_weekend,temperature,ground_type,equipment_quality,event,people_count,season,weather
0,0,1,11,2023,1,0,6.720739,футбольная,2,0,0,Spring/Autumn,ливень
1,1,1,11,2023,1,0,7.055569,футбольная,2,0,0,Spring/Autumn,облачно
2,2,1,11,2023,1,0,12.486396,футбольная,2,0,0,Spring/Autumn,солнечно
3,3,1,11,2023,1,0,11.386012,футбольная,2,0,0,Spring/Autumn,облачно
4,4,1,11,2023,1,0,10.342226,футбольная,2,0,0,Spring/Autumn,облачно


In [9]:
# Явно зададим, какие признаки являются категориальными
cat_features = ['ground_type',"is_weekend" ,'season', 'weather','hour','day','month','year','event',"ground_id"]


# Разделим данные на X (признаки) и y (целевую переменную)
X = df.drop(columns=['people_count'])
y = df['people_count']

from sklearn.model_selection import train_test_split

# Разделим данные на обучающую и временную выборки (80%/20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Теперь разделим временную выборку на валидационную и тестовую (50%/50%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Проверим размеры выборок
print(f"X_train shape: {X_train.shape}, X_val shape: {X_val.shape}, X_test shape: {X_test.shape}")


X_train shape: (907872, 12), X_val shape: (113484, 12), X_test shape: (113484, 12)


In [10]:
model = CatBoostRegressor(iterations=1000,  # Количество итераций
                          learning_rate=0.1,  # Скорость обучения
                          depth=10,  # Глубина деревьев
                          cat_features=cat_features,  # Категориальные признаки
                          random_seed=42,  # Для воспроизводимости
                          loss_function='RMSE',  # Функция потерь
                          eval_metric='RMSE',  # Оценка метрики
                          verbose=50,  # Выводить каждые 100 итераций
                          early_stopping_rounds=50)  # Остановить через 50 итераций без улучшений


model.fit(X_train, y_train, eval_set=(X_val, y_val))

0:	learn: 2.7930253	test: 2.7942702	best: 2.7942702 (0)	total: 1.27s	remaining: 21m 12s
50:	learn: 0.7332249	test: 0.7237747	best: 0.7237747 (50)	total: 52.5s	remaining: 16m 16s
100:	learn: 0.7246594	test: 0.7187470	best: 0.7187470 (100)	total: 1m 44s	remaining: 15m 31s
150:	learn: 0.7221681	test: 0.7183029	best: 0.7182921 (146)	total: 2m 32s	remaining: 14m 20s
200:	learn: 0.7201500	test: 0.7180023	best: 0.7179781 (199)	total: 3m 27s	remaining: 13m 45s
250:	learn: 0.7183856	test: 0.7179006	best: 0.7179006 (250)	total: 4m 27s	remaining: 13m 17s
300:	learn: 0.7165317	test: 0.7179791	best: 0.7178981 (252)	total: 5m 26s	remaining: 12m 37s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7178980693
bestIteration = 252

Shrink model to first 253 iterations.


In [12]:
model.save_model('catboost_model.cbm')